In [24]:
!pip install pandas numpy gensim scikit-learn tensorflow keras


In [41]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split, GridSearchCV,  ParameterGrid
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from multiprocessing import Pool

# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')


# Assuming your DataFrame has columns 'text' for the text and 'difficulty' for the labels
texts = training_data_df['sentence']
labels = training_data_df['difficulty']


# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
X = tfidf_vectorizer.fit_transform(texts).toarray()

In [43]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)
y_categorical = to_categorical(y_encoded)

In [44]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Select a random subset of 1000 samples for hyperparameter tuning
subset_indices = np.random.choice(X_train.shape[0], 1000, replace=False)
X_train_subset = X_train[subset_indices]
y_train_subset = y_train[subset_indices]

# Define the model and parameters for grid search
model = GradientBoostingClassifier()
parameters = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}


# Define a function to train a model with a given set of hyperparameters
def train_model(params):
    model = GradientBoostingClassifier(**params)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score, params

# Define your hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    # Add other parameters here
}

# Create a list of all parameter combinations
param_list = list(ParameterGrid(param_grid))

# Use multiprocessing to train models in parallel
with Pool(processes=4) as pool:  # Adjust the number of processes based on your CPU
    results = pool.map(train_model, param_list)

# Find the best performing set of parameters
best_score, best_params = max(results, key=lambda x: x[0])

print(f"Best Score: {best_score}")
print(f"Best Parameters: {best_params}")


Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Ce

KeyboardInterrupt: 

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

gbdt_model = GradientBoostingClassifier()
gbdt_model.fit(X_train, y_train)
y_pred_gbdt = gbdt_model.predict(X_test)

# Evaluate GBDT Model
print("GBDT Model Accuracy:", accuracy_score(y_test, y_pred_gbdt))
print(classification_report(y_test, y_pred_gbdt))

GBDT Model Accuracy: 0.39791666666666664
              precision    recall  f1-score   support

           0       0.46      0.64      0.54       166
           1       0.30      0.27      0.29       158
           2       0.38      0.30      0.33       166
           3       0.34      0.33      0.34       153
           4       0.35      0.36      0.36       152
           5       0.50      0.47      0.48       165

    accuracy                           0.40       960
   macro avg       0.39      0.40      0.39       960
weighted avg       0.39      0.40      0.39       960



In [33]:

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_categorical[np.where(y_encoded == y_train)[0]], epochs=10, batch_size=32, validation_split=0.1)

y_pred_dnn = model.predict(X_test)

# Evaluate DNN Model
print("DNN Model Accuracy:", accuracy_score(y_test, np.argmax(y_pred_dnn, axis=1)))
print(classification_report(y_test, np.argmax(y_pred_dnn, axis=1)))

ValueError: operands could not be broadcast together with shapes (4800,) (3840,) 

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# download the data from the given URLs and store them in dataframes
sample_submission_df = pd.read_csv('sample_submission.csv')
training_data_df = pd.read_csv('training_data.csv')
unlabelled_test_data_df = pd.read_csv('unlabelled_test_data.csv')

# Assuming your DataFrame has columns 'sentence' for the text and 'difficulty' for the labels
texts = training_data_df['sentence']
labels = training_data_df['difficulty']

# TF-IDF Vectorization with bi-grams
tfidf_vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1, 2))
X = tfidf_vectorizer.fit_transform(texts).toarray()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert y_train to categorical for DNN
y_train_categorical = to_categorical(y_train, num_classes=len(np.unique(y_encoded)))

# Gradient Boosting Classifier with more estimators
gbdt_model = GradientBoostingClassifier(n_estimators=200)
gbdt_model.fit(X_train, y_train)
y_pred_gbdt = gbdt_model.predict(X_test)

# Evaluate GBDT Model
print("GBDT Model Accuracy:", accuracy_score(y_test, y_pred_gbdt))
print(classification_report(y_test, y_pred_gbdt))

# Deep Neural Network Model with adjusted layers
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train_categorical, epochs=20, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

# Predictions
y_pred_dnn = model.predict(X_test)
y_pred_dnn_classes = np.argmax(y_pred_dnn, axis=1)

# Evaluate DNN Model
print("DNN Model Accuracy:", accuracy_score(y_test, y_pred_dnn_classes))
print(classification_report(y_test, y_pred_dnn_classes))

GBDT Model Accuracy: 0.39895833333333336
              precision    recall  f1-score   support

           0       0.45      0.66      0.54       166
           1       0.35      0.32      0.34       158
           2       0.36      0.27      0.30       166
           3       0.35      0.30      0.32       153
           4       0.34      0.39      0.36       152
           5       0.51      0.45      0.48       165

    accuracy                           0.40       960
   macro avg       0.39      0.40      0.39       960
weighted avg       0.39      0.40      0.39       960

Epoch 1/20
54/54 [==============================] - 2s 31ms/step - loss: 1.6999 - accuracy: 0.2749 - val_loss: 1.4735 - val_accuracy: 0.4193
Epoch 2/20
54/54 [==============================] - 2s 30ms/step - loss: 1.0531 - accuracy: 0.6256 - val_loss: 1.3462 - val_accuracy: 0.4740
Epoch 3/20
54/54 [==============================] - 2s 29ms/step - loss: 0.3772 - accuracy: 0.8889 - val_loss: 1.6178 - val_accuracy: 